<a href="https://colab.research.google.com/github/Shubh3005/NittanyAIMLB/blob/main/FineTuningMLB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-22 22:18:20--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.207, 173.194.203.207, 74.125.199.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   155MB/s    in 0.4s    

2024-11-22 22:18:21 (155 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [7]:
from PIL import Image
import numpy as np


cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")


# Initialize the lists
x_train = []
y_train = []

# Set desired image dimensions
desired_width, desired_height = 128, 128

# Loop through the first 100 images alternately from cats and dogs
i = 0
while i < 100:
    if i % 2 == 0:  # Even index: Cat image
        cat_images = sorted(os.listdir(cats_dir))
        im = Image.open(os.path.join(cats_dir, cat_images[i])).convert("RGB")
    else:  # Odd index: Dog image
        dog_images = sorted(os.listdir(dogs_dir))
        im = Image.open(os.path.join(dogs_dir, dog_images[i])).convert("RGB")

    # Resize and append to training data
    im_resized = im.resize((desired_width, desired_height))
    x_train.append(np.array(im_resized))

    # Append label
    y_train.append(1 if i % 2 == 0 else 0)

    i += 1

In [8]:

import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Convert lists to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

# Normalize image data
x_train = x_train / 255.0

# Split into training and validation sets
train_split = int(0.8 * len(x_train))
x_val, y_val = x_train[train_split:], y_train[train_split:]
x_train, y_train = x_train[:train_split], y_train[:train_split]

# Adjust input dimensions to match ResNet50 input
x_train_resized = tf.image.resize(x_train, [150, 150])
x_val_resized = tf.image.resize(x_val, [150, 150])

# Define the model
model = Sequential([
    pretrained_model,
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    x_train_resized, y_train,
    validation_data=(x_val_resized, y_val),
    epochs=10,
    batch_size=16
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(x_val_resized, y_val, verbose=2)
print(f"Validation Accuracy: {val_accuracy:.2f}")

# Make predictions
predictions = model.predict(x_val_resized)
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Show some predictions
for i in range(5):
    print(f"True label: {y_val[i]}, Predicted label: {predicted_classes[i]}")


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 70s 8s/step - accuracy: 0.4750 - loss: 0.8769 - val_accuracy: 0.5000 - val_loss: 0.7053
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 30s 6s/step - accuracy: 0.8625 - loss: 0.2763 - val_accuracy: 0.5000 - val_loss: 0.7095
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 37s 6s/step - accuracy: 1.0000 - loss: 0.0496 - val_accuracy: 0.5000 - val_loss: 0.7046
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 26s 6s/step - accuracy: 1.0000 - loss: 0.0332 - val_accuracy: 0.5000 - val_loss: 0.7035
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 42s 6s/step - accuracy: 1.0000 - loss: 0.0524 - val_accuracy: 0.5000 - val_loss: 0.7002
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 1.0000 - loss: 0.0185 - val_accuracy: 0.5000 - val_loss: 0.6981
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 40s 6s/step - accuracy: 1.0000 - loss: 0.0076 - val_accuracy: 0.5000 - val_loss: 0.6956
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 42s 7s/step - accuracy: 1.0000 - loss: 0.0071 - val_accuracy: 0.5000 - val_loss: 0.6937
Epoch 9/